# Jane Street Market Prediction
[Kaggle Jane Street Market Prediction](https://www.kaggle.com/c/jane-street-market-prediction/overview)


In [1]:
import pandas as pd
import numpy as np
import datatable as dt
X = dt.fread('jane-street-market-prediction/train.csv').to_pandas()

In [2]:
Xt = X[X.date >= 460]
yt = np.where((Xt.resp > 0), 1, 0)
X = X[(X.date > 85) & (X.date < 460)]
y = np.where((X.resp > 0), 1, 0)
drop_cols = ['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id']
X.drop(drop_cols, axis=1, inplace=True)
Xt.drop(drop_cols, axis=1, inplace=True)
yXt = pd.concat([pd.Series(yt), Xt], axis=1)

In [3]:
yXt

,0,weight,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390486,NaN,0.000000,1.0,-1.649365,-1.169996,-0.889129,-1.256179,-0.265419,-0.383478,0.526201,...,-2.421753,-1.896874,-1.260055,1.947725,-1.994399,-1.685163,-2.866165,-0.216130,-1.892048,0.901585
2390487,NaN,0.000000,1.0,2.432943,5.284504,-0.337469,-0.494263,-0.442409,-0.739016,-0.064645,...,-0.677511,-0.936553,1.064936,3.119762,-0.419796,-0.208975,-0.146749,0.730166,0.648452,2.068737
2390488,NaN,0.000000,1.0,-0.622475,-0.963682,0.532835,0.392287,0.977046,0.819693,0.140248,...,-0.459167,-2.956745,-0.640334,-2.279663,-0.950259,-4.388417,-1.669922,-3.288939,-1.336142,-2.814239
2390489,NaN,0.283405,-1.0,-1.463757,-1.107228,-2.286985,-3.156451,-1.690676,-2.348199,-0.683812,...,-2.651236,-2.035894,-1.780962,0.881246,-2.202140,-1.912601,-3.341684,-0.571188,-2.185795,0.627452


In [4]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(Xt, yt, reference=lgb_train)


In [5]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [6]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.070392 seconds.
You can set `force_col_wise=true` to remove the overhead.
[1]	valid_0's binary_logloss: 0.693133	valid_0's auc: 0.51627
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.693052	valid_0's auc: 0.515838
[3]	valid_0's binary_logloss: 0.692973	valid_0's auc: 0.518485
[4]	valid_0's binary_logloss: 0.692895	valid_0's auc: 0.51974
[5]	valid_0's binary_logloss: 0.69283	valid_0's auc: 0.520601
[6]	valid_0's binary_logloss: 0.692754	valid_0's auc: 0.520788
[7]	valid_0's binary_logloss: 0.69274	valid_0's auc: 0.520858
[8]	valid_0's binary_logloss: 0.692691	valid_0's auc: 0.520898
[9]	valid_0's binary_logloss: 0.692652	valid_0's auc: 0.521208
[10]	valid_0's binary_logloss: 0.692612	valid_0's auc: 0.521453
[11]	valid_0's binary_logloss: 0.692577	valid_0's auc: 0.522066
[12]	valid_0's binary_logloss: 0.692527	valid_0's auc: 0.522971
[13]	valid_0's binar

In [7]:
y_pred = gbm.predict(Xt, num_iteration=gbm.best_iteration)

LightGBMError: The number of features in data (132) is not the same as it was in training data (131).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import mean_squared_error
print('The rmse of prediction is:', mean_squared_error(yt, y_pred) ** 0.5)